In [1]:
import random
from typing import Optional

import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import models, datasets, transforms

from aimet_torch.adaround.adaround_weight import AdaroundParameters
from aimet_torch.auto_quant import AutoQuant
# from aimet_torch.auto_quant_v2 import AutoQuant



2025-02-20 16:01:39,918 - root - INFO - AIMET


c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\onnxscript\converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
<frozen abc>:106: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.


In [ ]:


# Step 1. Define constants and helper functions
EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 2000
BATCH_SIZE = 32

_subset_samplers = {}

def _create_sampled_data_loader(dataset, num_samples):
    if num_samples not in _subset_samplers:
        indices = random.sample(range(len(dataset)), num_samples)
        _subset_samplers[num_samples] = SubsetRandomSampler(indices=indices)
    return DataLoader(dataset,
                      sampler=_subset_samplers[num_samples],
                      batch_size=BATCH_SIZE)

# Step 2. Prepare model and dataset
import MobileNetV2Pretrained
import os
import sys

fp32_model = MobileNetV2Pretrained.PreprocessedMobileNetV2(64, 'model/mobilenet_v2_coco.pth')

fp32_model.eval()

input_shape = (1, 3, 224, 224)
dummy_input = torch.randn(input_shape)

transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.unsqueeze(0).to("cuda"))
    ])

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT_DIR)

from dataset import DatasetReader
from dataset import utils as dsutils

datasetCOCO = DatasetReader.COCODataset(
    annotation_file = r"C:/Users/Pro/Documents/___FAKULTET___/LPCV/LPCV_2025_T1/data/annotations/annotations/instances_val2017.json", 
    image_dir= r'C:/Users/Pro/Documents/___FAKULTET___/LPCV/LPCV_2025_T1/data/val2017/val2017',
    target_classes=[s.lower() for s in dsutils.GLOBAL_CLASSES],
    transform = transform,
    )

# NOTE: In the actual use cases, a real dataset should provide by the users.
eval_dataset = datasetCOCO

print(len(eval_dataset))

# import matplotlib.pyplot as plt
# plt.imshow(eval_dataset[601][0].permute(1, 2, 0))
# plt.show()


23043


In [3]:

# Step 3. Prepare unlabeled dataset
# NOTE: In the actual use cases, the users should implement this part to serve
#       their own goals if necessary.
class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset):
        self._dataset = dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index):
        images, _ = self._dataset[index]
        return images

unlabeled_dataset = UnlabeledDatasetWrapper(eval_dataset)
unlabeled_data_loader = _create_sampled_data_loader(unlabeled_dataset, CALIBRATION_DATASET_SIZE)

# Step 4. Prepare eval callback
# NOTE: In the actual use cases, the users should implement this part to serve
#       their own goals if necessary.
def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = len(eval_dataset)

    eval_data_loader = _create_sampled_data_loader(eval_dataset, num_samples)

    num_correct_predictions = 0
    for images, labels in eval_data_loader:
        predictions = torch.argmax(model(images.cuda()), dim=1)
        num_correct_predictions += torch.sum(predictions.cpu() == labels)

    return int(num_correct_predictions) / num_samples



In [4]:


# Step 5. Create AutoQuant object
auto_quant = AutoQuant(fp32_model.cuda(),
                       dummy_input.cuda(),
                       unlabeled_data_loader,
                       eval_callback)


In [5]:

# Step 6. (Optional) Set adaround params
ADAROUND_DATASET_SIZE = 2000
adaround_data_loader = _create_sampled_data_loader(unlabeled_dataset, ADAROUND_DATASET_SIZE)
adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader))
auto_quant.set_adaround_params(adaround_params)


In [6]:
fp32_model(eval_dataset[0][0].cuda())

ValueError: expected 4D input (got 3D input)

In [7]:
traced = torch.jit.trace(fp32_model, dummy_input.cuda())

In [8]:

# Step 7. Run AutoQuant
sim, initial_accuracy = auto_quant.run_inference()
print(f"- Quantized Accuracy (before optimization): {initial_accuracy:.4f}")



- Prepare Model


Traceback (most recent call last):
  File "c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\aimet_torch\_base\auto_quant.py", line 879, in run_inference
    model = sess.wrap(self._prepare_model)(self.fp32_model)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\aimet_torch\_base\auto_quant.py", line 475, in wrapper
    ret = fn(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\aimet_torch\_base\auto_quant.py", line 1063, in _prepare_model
    prepared_model = prepare_model(model, **self._model_preparer_kwargs)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Pro\Documents\___FAKULTET___\LPCV\LPCV_2025_T1\LPCV_2025_T1\venv\Lib\site-packages\aimet_torch\model_preparer.py", line 466, in prepare_model
 

NameError: module is not installed as a submodule

In [ ]:

model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy:.4f}")